<a href="https://colab.research.google.com/github/vonhuhoanghuy/ChatbotAsisstant/blob/main/trainChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
   import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
pip install numpy tensorflow nltk pickle-mixin

In [ ]:
import json
import random
import numpy as np

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import pickle

In [ ]:
# Tải dữ liệu từ file data.json
lemmatizer = WordNetLemmatizer()
with open('data.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)

words, classes, documents = [], [], []
ignore_words = ['?', '!', '.', ',']

In [ ]:
# Xử lý dữ liệu từ file JSON
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
# Tạo dữ liệu huấn luyện
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = [lemmatizer.lemmatize(w.lower()) for w in doc[0]]

    # Đảm bảo tất cả các túi từ (bag) có cùng kích thước với chiều dài của từ điển words
    for w in words:
        bag.append(1 if w in word_patterns else 0)

    # Tạo một hàng kết quả cho mỗi nhãn
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Thêm túi từ và hàng kết quả vào training
    training.append([bag, output_row])

# Chuyển đổi `training` thành mảng numpy với định dạng đồng nhất
training = np.array(training, dtype=object)
train_x = np.array([i[0] for i in training], dtype=np.float32)
train_y = np.array([i[1] for i in training], dtype=np.float32)


In [ ]:
# Xây dựng mô hình mạng nơ-ron
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Biên dịch mô hình
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
# Huấn luyện mô hình
model.fit(train_x, train_y, epochs=500, batch_size=5, verbose=1)
model.save('chatbot_model.h5')

print("Huấn luyện hoàn tất và mô hình đã được lưu.")


Epoch 1/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.0348 - loss: 3.5749
Epoch 2/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0239 - loss: 3.5149
Epoch 3/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0859 - loss: 3.4318
Epoch 4/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2106 - loss: 3.2365
Epoch 5/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1264 - loss: 3.1451
Epoch 6/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1893 - loss: 2.8826
Epoch 7/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2486 - loss: 2.6551
Epoch 8/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3443 - loss: 2.3514
Epoch 9/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3086 - loss: 2.2733
Epoch 10/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3405 - loss: 2.0515
Epoch 11/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5969 - loss: 1.5784
Epoch 12/500
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy

Huấn luyện hoàn tất và mô hình đã được lưu.


In [ ]:
import json
import random
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

# Tải các gói dữ liệu cần thiết
nltk.download('punkt')
nltk.download('wordnet')

# Tải mô hình và dữ liệu đã lưu
model = load_model('chatbot_model.h5')
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Tải dữ liệu từ file data.json
lemmatizer = WordNetLemmatizer()
with open('data.json', 'r', encoding='utf-8') as file:
    intents_data = json.load(file) # Renamed to intents_data to avoid conflict

# Hàm để chuyển đổi câu hỏi từ người dùng thành dạng mà mô hình có thể xử lý
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    bag = [0] * len(words)
    sentence_words = clean_up_sentence(sentence)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_data): # Added intents_data as argument
    if intents_list:  # Kiểm tra nếu có nhãn nào được dự đoán
        intent = intents_list[0]['intent']
        for i in intents_data['intents']: # Access intents_data instead of intents
            if i['tag'] == intent:
                response = random.choice(i['responses'])
                return response
    return "Xin lỗi, tôi không hiểu câu hỏi của bạn."

# Vòng lặp để tương tác với người dùng
print("Chatbot đang hoạt động! (Gõ 'exit' để thoát)")
while True:
    user_input = input("Bạn: ")
    if user_input.lower() == 'exit':
        break
    intents = predict_class(user_input)
    response = get_response(intents, intents_data) # Pass intents_data to get_response
    print("Chatbot:", response)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Chatbot đang hoạt động! (Gõ 'exit' để thoát)
Bạn: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
Chatbot: Chào bạn, hôm nay bạn cần hỗ trợ gì?
